In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from urllib.parse import urlparse

In [2]:
url_base='https://www.ylh.gov.tw'
url_work='/?aid=104'
url_full=url_base+url_work
g=requests.get(url_full)
soup=BeautifulSoup(g.content, 'html.parser')

# Get Pages link

In [3]:
pages_link=[]
for link in soup.find_all('li',class_="page-item"):
    if link.find("a"):
        if link.find("a") == -1 or link.text == "第一頁" or link.text == "«" or link.text == "»":
            pass
        elif link.text == "最末頁":
            pages_link.append(url_base+link.find('a').get('href'))
        elif link.find('a').has_attr("disabled"):
            print('skip:',link.text)
        else:
            pages_link.append(url_base+link.find('a').get('href'))
pages_link

skip: 1
skip: ...


['https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=2',
 'https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=3',
 'https://www.ylh.gov.tw/?aid=104&pid=0&page_name=list&pageNo=4']

# Get work table

In [4]:
soup.find_all('table')[0].find('tbody')

<tbody>
<tr>
<td class="align-middle" data-title="標題" width="50%">公告本院護理部公開甄選院聘護理師(手術室)儲備20名(隨到隨招，請於上班時間先行電話預約)</td>
<td class="text-center align-middle" data-title="公告日期">2022-01-18</td>
<td class="text-center align-middle" data-title="截止日期">2022-07-31</td>
<td class="text-center align-middle" data-title="下載">
<a href="/public/jobnews/4eb1cf3e5ea8a5858cb8f631988ab5a8.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="[PDF]下載  (另開新視窗) ">查看</a>
</td>
</tr>
<tr>
<td class="align-middle" data-title="標題" width="50%">公告本院護理部公開甄選院聘護理師/實習護士儲備30名(隨到隨招，請於上班時間先行電話預約)</td>
<td class="text-center align-middle" data-title="公告日期">2021-12-23</td>
<td class="text-center align-middle" data-title="截止日期">2022-07-31</td>
<td class="text-center align-middle" data-title="下載">
<a href="/public/jobnews/ba6bb249a04a65caa2026732b239f9bb.pdf" rel="noopener noreferrer" style="color: blue;" target="_blank" title="[PDF]下載  (另開新視窗) ">查看</a>
</td>
</tr>
<tr>
<td class="align-middle" data-title

In [5]:
def get_each_page_wrok_table(soup,work_table):
    for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
        all_td = work.find_all("td")
        title = all_td[0].text
        place = '察看連結的簡章'
        deadline = all_td[2].text
        
        link=url_base+all_td[3].find('a').get('href')
        if "錄取名單" in title:
            print('skip: ',title)
        elif "甄試名單" in title:
            print('skip: ',title)
        elif "複試名單" in title:
            print('skip: ',title)
        elif "志願服務工作" in title:
            print('skip: ',title)
        elif "甄審會" in title:
            print('skip: ',title)
        elif "時薪人員" in title:
            print('skip: ',title)
        else:
            originization = re.search("\B院((.*)[室,部,中心])",title).group(1)
            new_title = re.search("\B聘((.*)[員,師,廚])",title).group(1)
            print("召聘職稱",new_title,'院區',place,"截止日期",deadline,"職缺單位",originization,"links",link)
            work_table.append([new_title, deadline, originization, place, link ])
    return work_table

In [6]:
def get_base_web_data(url_base,url_work_table=None):
    if url_work_table != None:
        url = url_base + url_work_table
    else:
        url = url_base
    g=requests.get(url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [8]:
work_table = []
for page in pages_link:
    soup = get_base_web_data(page)
    work_table = get_each_page_wrok_table(soup,work_table)

skip:  公告本院護理部111年5月6日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年5月4日公開甄選院聘護理師/實習護士儲備30名錄取名單
召聘職稱 助廚 院區 察看連結的簡章 截止日期 2022-12-31 職缺單位 營養室 links https://www.ylh.gov.tw/public/jobnews/300f01ae03a2571529b5b1f21e968326.pdf
召聘職稱 照顧服務員 院區 察看連結的簡章 截止日期 2022-12-31 職缺單位 護理部 links https://www.ylh.gov.tw/public/jobnews/2a78c957732fe3c5a6dc5791a081da3c.pdf
召聘職稱 助理管理師 院區 察看連結的簡章 截止日期 2022-06-23 職缺單位 安全衛生室111年5月4日公開甄選院聘助理管理師(身心 links https://www.ylh.gov.tw/public/jobnews/075d3c7f9f84898c8f932d7340d6ca11.pdf
召聘職稱 管理師 院區 察看連結的簡章 截止日期 2022-06-10 職缺單位 企劃管理室 links https://www.ylh.gov.tw/public/jobnews/753fdd4238b4b67158f1d5c504a2db4e.pdf
skip:  公告本院111/5/4甄審會公開甄選複試結果(公職醫事人員類)
skip:  公告本院護理部(病房)111年4月26日公開甄選院聘助理管理師1名「錄取名單」
skip:  公告本院檢驗醫學部111年5月5日公開甄選院聘助理管理師職務代理人1名「錄取名單」
skip:  公告本院護理部(護理之家)111年4月28日公開甄選院聘照顧服務員1名「錄取名單」
skip:  公告本院護理部(門診)111年4月22日甄選院聘護理師2名錄取名單
skip:  公告本院護理部111年5月3日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年4月26日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本院護理部111年4月22日公開甄選院聘護理師/實習護士儲備30名錄取名單
skip:  公告本

In [111]:
re.search("\B院((.*)部)","公告本院護理部公開甄選院聘護理師(手術室)儲備").group(1)

'護理部'

In [110]:
re.search("\B聘((.*)師)","公告本院護理部公開甄選院聘護理師(手術室)儲備").group(1)

'護理師'

In [112]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" , '院區','連結'])

print(work_table)

        召聘職稱          期限               職缺單位       院區  \
0        護理師  2022-07-31   護理部公開甄選院聘護理師(手術室  察看連結的簡章   
1        護理師  2022-07-31                護理部  察看連結的簡章   
2     契約主治醫師  2022-05-20           環境及職業醫學部  察看連結的簡章   
3      助理管理師  2022-05-09  主計室公開甄選院聘助理管理師(身心  察看連結的簡章   
4       副管理師  2022-05-10                護理部  察看連結的簡章   
5      醫事檢驗師  2022-05-20                病理部  察看連結的簡章   
6        佐理員  2022-05-09            護理部(供應室  察看連結的簡章   
7         助廚  2022-12-31                營養室  察看連結的簡章   
8        護理師  2022-07-31   護理部公開甄選院聘護理師(手術室  察看連結的簡章   
9        護理師  2022-07-31                護理部  察看連結的簡章   
10    契約主治醫師  2022-05-20           環境及職業醫學部  察看連結的簡章   
11     助理管理師  2022-05-09  主計室公開甄選院聘助理管理師(身心  察看連結的簡章   
12      副管理師  2022-05-10                護理部  察看連結的簡章   
13     醫事檢驗師  2022-05-20                病理部  察看連結的簡章   
14       佐理員  2022-05-09            護理部(供應室  察看連結的簡章   
15  資深/專科護理師  2022-06-15            醫療部/護理部  察看連結的簡章   

                                               